## A simple example on training a CNN model with cifar10 data set.

### Set up keras

Import TensorFlow into your program to get started:

In [1]:
from tensorflow import keras
import numpy as np

In [2]:
import mlrun
import mlrun.frameworks.tf_keras as mlrun_tf_keras

### Load a dataset

Load the dataset

In [3]:
def get_datasets(batch_size: int=64):

    # Load the data:
    (train_x, train_y), (test_x, test_y) = keras.datasets.cifar10.load_data()

    # Scale the images:
    train_x = train_x.astype(np.float32)
    train_x /= 255.0
    test_x = test_x.astype(np.float32)
    test_x /= 255.0

    # One hot encode target values
    train_y = keras.utils.to_categorical(train_y)
    test_y = keras.utils.to_categorical(test_y)

    # create data generator
    image_data_generator = keras.preprocessing.image.ImageDataGenerator(
        width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True
    )

    # Wrap up the datasets:
    training_set = image_data_generator.flow(train_x, train_y, batch_size=batch_size)
    validation_set = (test_x, test_y)

    return training_set, validation_set

### Build a machine learning model

Build a `keras.models.Sequential` model by stacking layers.

In [4]:
def get_model():
    # Initialize a sequential model:
    model = keras.models.Sequential()

    # Add the input block:
    model.add(keras.layers.InputLayer(input_shape=(32, 32, 3)))

    # Add VGG blocks:
    filters_list = [32, 64, 128]
    dropout_list = [0.3, 0.5, 0.5]
    for filters, dropout in zip(filters_list, dropout_list):
        model.add(keras.layers.Conv2D(filters, (3, 3), padding="same", activation="relu"))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Conv2D(filters, (3, 3), padding="same", activation="relu"))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(keras.layers.Dropout(dropout))

    # Add the output block:
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(filters_list[-1], activation="relu"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(dropout_list[-1]))
    model.add(keras.layers.Dense(10, activation="softmax"))

    return model

### Define training function

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [5]:
def train(batch_size: int=64, lr: float=0.001, epochs: int = 3):
    
    print(f'batch size    === >>> {batch_size}')
    print(f'learning rate === >>> {lr}')
    print(f'epochs        === >>> {epochs}')
    
    # Get the datasets:
    training_set, validation_set = get_datasets(batch_size)

    # Get the model:
    model = get_model()

    ################### Apply MLRun here: ###################
    context = mlrun.get_or_create_ctx(name="cifar10_ctx")
    mlrun_tf_keras.apply_mlrun(model=model, context=context)
    ################### Apply MLRun here: ###################
    
    # Compile the model:
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )

    # Train the model:
    model.fit(training_set, epochs=epochs, validation_data=validation_set)

### Now you dno't train your model by running the train() function in this nodebook

In [6]:
# train()